In [1]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.layers.core import Dense,Activation,Dropout,Flatten
from keras.utils import np_utils
from keras import *
import tensorflow as tf
from keras.optimizers import SGD
import os
from six.moves import cPickle as pickle
import platform
import numpy as np
 
def load_files(filenames):
    data = np.array([])
    labels = np.array([])
    for name in filenames:
        with open(name, 'rb') as f:
            mydict = pickle.load(f, encoding='latin1')

        # The labels have different names in the two datasets.
        label_func = lambda x: np.array(x['fine_labels'], dtype='int32')
        newlabels = label_func(mydict)
        if data.size:
            data = np.vstack([data, mydict['data']])
            labels = np.hstack([labels, newlabels])
        else:
            data = mydict['data']
            labels = newlabels
    data = np.reshape(data, [-1, 3, 32, 32], order='C')
    data = np.transpose(data, [0, 2, 3, 1])

    return data, labels

def load_CIFAR100(data_dir):
    """ load all of cifar """
    train_files = ['train']
    train_files = [os.path.join(data_dir, f) for f in train_files]
    test_files = ['test']
    test_files = [os.path.join(data_dir, f) for f in test_files]
    num_classes = 100
    train_data, train_labels = load_files(train_files)
    test_data, test_labels = load_files(test_files)

    return train_data, train_labels, test_data, test_labels
 
def get_CIFAR100_data(val_size=2000):
    # Load the raw CIFAR-100 data
    cifar100_dir = '/Users/taraapple/Desktop/cifar-100-python'
    x_train, y_train, x_test, y_test = load_CIFAR100(cifar100_dir)
    # Subsample the data
 
    x_train, x_val = np.split(x_train,
                              [x_train.shape[0]-val_size])
    y_train, y_val = np.split(y_train,
                              [y_train.shape[0]-val_size])
    
    x_train = x_train.astype('float32')
    x_val = x_val.astype('float32')
    x_test = x_test.astype('float32')
 
    x_train /= 255
    x_val /= 255
    x_test /= 255

    return x_train, y_train, x_val, y_val, x_test, y_test
 
# Invoke the above function to get our data.
x_train, y_train, x_val, y_val, x_test, y_test = get_CIFAR100_data()
 
print('Train data shape: ', x_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', x_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', x_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (48000, 32, 32, 3)
Train labels shape:  (48000,)
Validation data shape:  (2000, 32, 32, 3)
Validation labels shape:  (2000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [3]:
batch_size = 128
# building model with too many classes inially for simpler transfer learning afterwards
nb_classes = 100
nb_epoch = 12


# print shape of data while model is building
print("{1} train samples, {2} channel{0}, {3}x{4}".format("" if x_train.shape[1] == 1 else "s", *x_train.shape))
print("{1}  test samples, {2} channel{0}, {3}x{4}".format("" if x_test.shape[1] == 1 else "s", *x_test.shape))
print("{1}  val samples, {2} channel{0}, {3}x{4}".format("" if x_val.shape[1] == 1 else "s", *x_val.shape))
# input image dimensions
_, img_channels, img_rows, img_cols = x_train.shape

# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)
y_val = np_utils.to_categorical(y_val, nb_classes)

model = Sequential()
 
model.add(Conv2D(32,(3,3),padding='same',input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
model.add(Conv2D(64,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(100,activation='softmax'))
# let's train the model using SGD + momentum (how original).
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=128, epochs=12, verbose=1, validation_data=(x_val, y_val))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

48000 train samples, 32 channels, 32x3
10000  test samples, 32 channels, 32x3
2000  val samples, 32 channels, 32x3
Epoch 1/12
375/375 [==============================] - 69s 184ms/step - loss: 4.4579 - accuracy: 0.0304 - val_loss: 4.1327 - val_accuracy: 0.0745
Epoch 2/12
375/375 [==============================] - 69s 184ms/step - loss: 4.0024 - accuracy: 0.0858 - val_loss: 3.8193 - val_accuracy: 0.1335
Epoch 3/12
375/375 [==============================] - 69s 185ms/step - loss: 3.7316 - accuracy: 0.1272 - val_loss: 3.5078 - val_accuracy: 0.1745
Epoch 4/12
375/375 [==============================] - 68s 183ms/step - loss: 3.5091 - accuracy: 0.1664 - val_loss: 3.3296 - val_accuracy: 0.1955
Epoch 5/12
375/375 [==============================] - 69s 183ms/step - loss: 3.3187 - accuracy: 0.2018 - val_loss: 3.1525 - val_accuracy: 0.2310
Epoch 6/12
375/375 [==============================] - 69s 184ms/step - loss: 3.1460 - accuracy: 0.2340 - val_loss: 2.9369 - val_accuracy: 0.2815
Epoch 7/12
375/

In [4]:
model.fit(x_train, y_train, batch_size=128, epochs=12, verbose=1, validation_data=(x_val, y_val))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Epoch 1/12
375/375 [==============================] - 68s 180ms/step - loss: 2.3172 - accuracy: 0.3979 - val_loss: 2.3553 - val_accuracy: 0.4050
Epoch 2/12
375/375 [==============================] - 69s 185ms/step - loss: 2.2280 - accuracy: 0.4143 - val_loss: 2.3078 - val_accuracy: 0.4175
Epoch 3/12
375/375 [==============================] - 70s 187ms/step - loss: 2.1500 - accuracy: 0.4322 - val_loss: 2.2653 - val_accuracy: 0.4215
Epoch 4/12
375/375 [==============================] - 71s 190ms/step - loss: 2.0687 - accuracy: 0.4495 - val_loss: 2.2179 - val_accuracy: 0.4215
Epoch 5/12
375/375 [==============================] - 71s 189ms/step - loss: 1.9988 - accuracy: 0.4595 - val_loss: 2.1730 - val_accuracy: 0.4430
Epoch 6/12
375/375 [==============================] - 71s 189ms/step - loss: 1.9317 - accuracy: 0.4749 - val_loss: 2.1778 - val_accuracy: 0.4360
Epoch 7/12
375/375 [==============================] - 71s 190ms/step - loss: 1.8634 - accuracy: 0.4925 - val_loss: 2.1507 - val_ac